# MMIS671_Homework1 LP

**Parameters of the model**:

- $N$: number of Cable types
- $M$: number of resources
- $C[i]$: ***C***ost per foot for producing Cable $i$, for $i=1,...,N$.
- $P[i]$: ***P***rice per foot for purchasing Cable $i$, for $i=1,...,N$.
- $D[i]$: ***D***emand (minimum number of feet required) for Cable $i$, for $i=1,...,N$.
- $R[j][i]$: Number of minutes of ***R***esource $j$ needed to produce each foot of Cable $i$.
- $A[j]$: ***A***vailable number of minutes for resource $j$, for $j=1,...,M$.


**Decision variables**:
- $X[i]$: Number of feet of Cable $i$ produced, for $i=1,...,N$.
- $Y[i]$: Number of feet of Cable $i$ purchased, for $i=1,...,N$.

**Objective function**:
- $Minimize\ Cost = \sum_{i=1}^N(C[i]X[i]+P[i]Y[i])$

**Subject to constraints**:

- Demand constraints for Cables: $X[i]+Y[i] \ge D[i]$, for $i=1,...,N$.
- Availability constraints for resources: $\sum_{i=1}^N R[j][i]X[i] \le A[j]$, for $j=1,...,M$.

*All decision variables are non-negative*.

## Install PuLP
We shall use the PuLP library (https://pypi.org/project/PuLP/) to define and solve LP models.

In [1]:
! pip install pulp

## Import libraries

In [2]:
import pandas as pd # for data handling
from pulp import * # for LP model

## Get data
Read the *CSV* file containing LP parameters into a *pandas* dataframe.

In [3]:
DIR = '/content/drive/MyDrive/Colab Notebooks/courses/LP/examples/' # directory with CSV file with problem data
data_file = 'MMIS671_Homework1_data.csv' # CSV file with problem data
data = pd.read_csv(DIR + data_file) # read data into pandas dataframe
data # display dataframe

,Cable Type,A,B,C,Available hours
0,Demand (ft),60000.0,40000.0,120000.0,NaN
1,Production Cost/ft,6.0,8.0,10.0,NaN
2,Purchase Cost/ft,8.0,10.0,15.0,NaN
3,Drawing (mins/ft),0.1,0.2,0.1,400.0
4,Annealing (mins/ft),0.1,0.2,0.2,600.0
5,Stranding (mins/ft),0.1,0.3,0.3,800.0
6,Extrusion (mins/ft),0.1,0.3,0.1,500.0
7,Assembly (mins/ft),0.2,0.1,0.4,1000.0


## Specify LP parameters
Obtain LP parameters from *pandas* dataframe.



In [4]:
CABLES = list(data)[1:-1] # list of products
N = len(CABLES) # number of products
print(f'{N} CABLES: {CABLES}')

V = data.values # (8 x 5) array with LP parameter values

D = V[0].tolist()[1:-1] # D[i]: demand for product i
print(f'Demand for products: {D}') # show demand

C = V[1].tolist()[1:-1] # P[i]: production cost per unit for product i
print(f'Production cost per unit: {C}') # show production cost per unit

P = V[2].tolist()[1:-1] # P[i]: purchase price per unit for product i
print(f'Purchase price per unit: {P}') # show purchase price per unit


RESOURCES = [v.split(' ')[0] for v in V[:,0].tolist()][3:] # list of resources
M = len(RESOURCES) # number of resources
print(f'{M} resources: {RESOURCES}')

A = [v*60 for v in V[3:,-1].tolist()] # A[j] minutes of resource j available
R = V[3:,1:-1].tolist() # R[j][i]: number of units of resource j used per unit of product i

for j in range(M): # show data for each resource
    print(f'{RESOURCES[j]}: Units needed for production {R[j]}, Available: {A[j]}') 

3 CABLES: ['A', 'B', 'C']
Demand for products: [60000.0, 40000.0, 120000.0]
Production cost per unit: [6.0, 8.0, 10.0]
Purchase price per unit: [8.0, 10.0, 15.0]
5 resources: ['Drawing', 'Annealing', 'Stranding', 'Extrusion', 'Assembly']
Drawing: Units needed for production [0.1, 0.2, 0.1], Available: 24000.0
Annealing: Units needed for production [0.1, 0.2, 0.2], Available: 36000.0
Stranding: Units needed for production [0.1, 0.3, 0.3], Available: 48000.0
Extrusion: Units needed for production [0.1, 0.3, 0.1], Available: 30000.0
Assembly: Units needed for production [0.2, 0.1, 0.4], Available: 60000.0


## Create LP model
Create LP model, define decision variables, specify objective function and constraints.

In [5]:
title = 'homework1.lp' # name of problem
prob = LpProblem(title, LpMinimize) # Create LP model object for maximization

# Specify decision variables as non-negative
X = [LpVariable(f'x{CABLES[i]}', 0) for i in range(N)] # X[i]: feet of cable i produced
Y = [LpVariable(f'y{CABLES[i]}', 0) for i in range(N)] # X[i]: feet of cable i purchased
# add objective function to prob
prob += lpSum(C[i]*X[i] + P[i]*Y[i] for i in range(N))

# add demand constraints to prob
for i in range(N): # for each product i 
    prob += X[i] + Y[i] >= D[i], 'Demand_'+CABLES[i]

# add resource constraints to prob
for j in range(M): # for each resource j
    prob += sum(R[j][i]*X[i] for i in range(N)) <= A[j], RESOURCES[j] +'_availability'

prob.writeLP(title) # write model to LP file
print(open(title).read()) # display lp model

\* homework1.lp *\
Minimize
OBJ: 6 xA + 8 xB + 10 xC + 8 yA + 10 yB + 15 yC
Subject To
Annealing_availability: 0.1 xA + 0.2 xB + 0.2 xC <= 36000
Assembly_availability: 0.2 xA + 0.1 xB + 0.4 xC <= 60000
Demand_A: xA + yA >= 60000
Demand_B: xB + yB >= 40000
Demand_C: xC + yC >= 120000
Drawing_availability: 0.1 xA + 0.2 xB + 0.1 xC <= 24000
Extrusion_availability: 0.1 xA + 0.3 xB + 0.1 xC <= 30000
Stranding_availability: 0.1 xA + 0.3 xB + 0.3 xC <= 48000
End



## Solve LP model

In [6]:
prob.solve()
status = LpStatus[prob.status] # optimal found?
best_obj = value(prob.objective) # objective function value
opt_X = [x.varValue for x in X] # list with optimal quantities produced
opt_Y = [y.varValue for y in Y] # list with optimal quantities purchased
print("Solution: ", status)
print(f'Minimum cost = $ {best_obj:,.2f}') # show optimal cost
print("Optimal quantities")
opt_qty_df = pd.DataFrame([opt_X, opt_Y], columns=CABLES) # dataframe with optimal quantities
opt_qty_df.insert(0, 'quantity', ['produced', 'purchased'])
opt_qty_df.to_csv('optimal_quantity.csv', index=False) # save results in CSV file
opt_qty_df # show optimal quantities

Solution:  Optimal
Minimum cost = $ 1,936,000.00
Optimal quantities


,quantity,A,B,C
0,produced,48000.0,24000.0,120000.0
1,purchased,12000.0,16000.0,0.0


## Perform sensitivity analysis

In [7]:
sa = [] # results of sensitivity analysis
for name, c in prob.constraints.items():
    available = -c.constant # number of units available 
    used = available - c.slack # number of units used
    incr = -c.pi  # increase in profit if 1 additional unit of resource is available 
    sa.append([name, available, used, c.slack, incr]) 
    cols = ['resource', 'RHS', 'LHS', 'slack', 'cost decrease per additional unit']
sa_df = pd.DataFrame(sa, columns=cols).round(2) # dataframe with sensitivity analysis results
sa_df.to_csv('sensitivity_analysis.csv', index=False) # save results in CSV file
sa_df # show sensitivity analysis results

,resource,RHS,LHS,slack,cost decrease per additional unit
0,Demand_A,60000.0,60000.0,-0.0,-8.0
1,Demand_B,40000.0,40000.0,-0.0,-10.0
2,Demand_C,120000.0,120000.0,-0.0,-14.4
3,Drawing_availability,24000.0,21600.0,2400.0,-0.0
4,Annealing_availability,36000.0,33600.0,2400.0,-0.0
5,Stranding_availability,48000.0,48000.0,-0.0,4.0
6,Extrusion_availability,30000.0,24000.0,6000.0,-0.0
7,Assembly_availability,60000.0,60000.0,-0.0,8.0
